# Analyze 

In [70]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import datetime
import json
import folium
import utils

## Defines

In [2]:
path_nuts = Path('../export/masternuts.gpkg')
path_timedata = Path('../export/timedata.json')
path_docs = Path('../docs')

## Load data

In [3]:
# Load timedata
time_data = json.load(open(path_timedata, 'r', encoding='utf-8'))

# Load Nuts3
gpd_nuts_raw = gpd.read_file(path_nuts)

# Redice precision
gpd_nuts_raw['geometry'] = gpd_nuts_raw['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

## Experiment!

In [112]:
# Serialize JSON
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'name': x['name'],
    'earliest_sunrise_current_dt': x['sunrise']['current']['earliest'],
    'earliest_sunrise_summertime': x['sunrise']['summer']['earliest'],
    'earliest_sunrise_wintertime': x['sunrise']['winter']['earliest'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert current to only time like the others
def dt_to_time(dt):
    dt = datetime.datetime.strptime(dt.split('+')[0], '%Y-%m-%d %H:%M:%S.%f')
    return dt.time()

def time_to_time(dt):
    return datetime.datetime.strptime(dt, '%H:%M:%S.%f').time()

# Convert strings to time
df_timedata['earliest_sunrise_current'] = df_timedata['earliest_sunrise_current_dt'].apply(dt_to_time)
df_timedata['earliest_sunrise_summertime'] = df_timedata['earliest_sunrise_summertime'].apply(time_to_time)
df_timedata['earliest_sunrise_wintertime'] = df_timedata['earliest_sunrise_wintertime'].apply(time_to_time)

# Calculate difference between time_reference and given time
def calculate_difference_to_timereference(dt, time_reference):
    diff = datetime.datetime.combine(datetime.date.min, dt) - datetime.datetime.combine(datetime.date.min, time_reference)
    return round(diff.total_seconds() / 60)

def categorize_time_to_half_hour(dt):
    h = dt.hour
    if dt.minute < 30:
        return h
        return f"{h:02d}:00 - {h:02d}:30"
    else:
        return h + 0.5
        return f"{h:02d}:30 - {h+1:02d}:00"

df_timedata['earliest_sunrise_current_category'] = df_timedata['earliest_sunrise_current'].apply(
    categorize_time_to_half_hour
)

# df_timedata['earliest_sunrise_summertime_n'] = df_timedata['earliest_sunrise_summertime'].apply(
#     calculate_difference_to_timereference,
#     args=(df_timedata[df_timedata.name == 'Zürich'].iloc[0]['earliest_sunrise_summertime'],)
# )

# df_timedata['earliest_sunrise_wintertime_n'] = df_timedata['earliest_sunrise_wintertime'].apply(
#     calculate_difference_to_timereference,
#     args=(df_timedata[df_timedata.name == 'Zürich'].iloc[0]['earliest_sunrise_wintertime'],)
# )

# Merge Dataframes
gpd_nuts = gpd_nuts_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

In [40]:
#gpd_nuts = gpd_nuts_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

# Plot the filtered data
#gpd_nuts.plot(column='earliest_sunrise_current_n', cmap='RdBu')

In [ ]:
m = folium.Map(
    # tiles="cartodbpositron",
    tiles=None,
    # tiles="Cartodb dark_matter",
    prefer_canvas=True
)

# Create bins evenly distributed with 0 in the center
# min_value = gpd_nuts['earliest_sunrise_current_n'].min() - 1
# max_value = gpd_nuts['earliest_sunrise_current_n'].max() + 1
# bins = list(range(min_value, max_value + 1, (max_value - min_value) // 10))
# bins = sorted(set(bins + [-2,0, 2]))  # Ensure 0 is included and bins are unique and sorted

def add_choropleth(name, field, popup_fields):

    cp = folium.Choropleth(
        geo_data=gpd_nuts.to_json(default=str),
        name=name,
        data=df_timedata,
        columns=["nuts_id", field],
        key_on="feature.properties.NUTS_ID",
        fill_color="Greys",
        fill_opacity=1,
        line_opacity=0.2,
        # bins=bins,
        bins = 10,
        overlay=True,
    ).add_to(m)


    folium.GeoJsonTooltip(["name"] + popup_fields).add_to(cp.geojson)    

add_choropleth("Aktuelles System", 'earliest_sunrise_current_category', ['earliest_sunrise_current', 'earliest_sunrise_current_category'])
# add_choropleth("Sommerzeit", 'earliest_sunrise_summertime_n', ['earliest_sunrise_summertime', 'earliest_sunrise_summertime_n'])
# add_choropleth("Winterzeit", 'earliest_sunrise_wintertime_n', ['earliest_sunrise_wintertime', 'earliest_sunrise_wintertime_n'])

m.fit_bounds(m.get_bounds())

folium.LayerControl(collapsed=False).add_to(m)

# Add Title
# utils.addTitle(m, "Velounfälle (alle), nach Region geclustered")

m.save(path_docs / 'map.html')


In [52]:
bins

[-132, -110, -88, -66, -44, -22, 0, 22, 44, 66, 88]

In [30]:
test = gpd_nuts_raw.copy()
test['geometry'] = test['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

with open('test.txt', 'w') as f:
    f.write(test.to_json())